# Post Buy

In [1]:
import os
import time

import numpy as np
import scipy as sp
import bottleneck as bn

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

In [39]:
N = 1000
M = 10

pij = np.random.rand(N, M)
sj = np.random.rand(M)
sj = sj/np.sum(sj) + 0.02


In [40]:
lamb0 = np.random.rand(M) # initial lambda

In [12]:
def _func(lamb, pij, sj, N):
    """
    Lagrangian function
    """
    varphi = pij - lamb
    varphi_three = -bn.partition(-varphi, 3, -1)[:,0:3]
    return np.sum(varphi_three) + 3 * N * np.sum(lamb * sj)

In [74]:
_func(lamb0, pij, sj, N)

29553.063034076175

In [13]:
def _dfunc(lamb, pij, sj, N):
    """
    Lagrangian derivative
    """
    x = np.zeros((N, M))
    varphi = pij - lamb
    varphi_idx = bn.argpartition(-varphi, 3, -1)
    for i in range(N):
        for j in range(3):
            x[i, varphi_idx[i,j]] = 1
    return -np.sum(x, axis = 0) + 3 * sj * N

In [89]:
_dfunc(lamb0, pij, sj, N)

array([-30285.20276333,  12040.29256325, -21765.77948189, -21708.32454481,
        16428.39931215, -33440.22948796,  43565.74934935, -10946.10806747,
        38325.29200418,   7785.91111654])

In [14]:
def objective(pij, x):
    """
    Find the value of objective function given the dicision varaibles
    """
    return np.sum(pij * x)

In [15]:
def objective_lamb(lamb, pij):
    """
    Find the value of objective function given the Lagrangian multipiliers.
    """
    x = np.zeros((N, M))
    varphi = pij - lamb
    varphi_idx = bn.argpartition(-varphi, 3, -1)
    for i in range(N):
        for j in range(3):
            x[i, varphi_idx[i,j]] = 1
    return np.sum(pij * x)

In [22]:
# Check whether satifies the constraints or not.
def checkConstraint(lamb, pij, sj, N):
    """
    """
    x = np.zeros((N, M))
    varphi = pij - lamb
    varphi_idx = bn.argpartition(-varphi, 3, -1)
    for i in range(N):
        for j in range(3):
            x[i, varphi_idx[i,j]] = 1
    return np.sum(x, axis = 0) - 3*sj*N

In [ ]:
# 随机生成 x[i][j]


In [32]:
def adam(theta0, pij, sj, N, verbose = True):
    """
    """
    if verbose:
        start = time.time()
    
    alpha, beta1, beta2, eps = 0.001, 0.9, 0.999, 1e-8
    beta1powert, beta2powert = 1.0, 1.0
    
    nitermax = 10000
    niter = 0
    
    theta_old = theta0 
    ndim = len(theta0)
    mold = np.zeros(ndim)
    vold = np.zeros(ndim)
    
    tolx = 1e-5
    tolf = 1e-5
    
    fold = _func(theta0, pij, sj, N)
    
    while niter < nitermax:
        
        print("iteration: {0}".format(niter))
        
        niter += 1
        
        g = _dfunc(theta_old, pij, sj, N)
        mnew = beta1 * mold + (1-beta1)*g
        vnew = beta2 * vold + (1-beta2)*g*g
        
        beta1powert *= beta1
        beta2powert *= beta2
        
        mhat = mnew/(1 - beta1powert)
        vhat = vnew/(1 - beta2powert)
        
        theta_new = theta_old - alpha * mhat / (np.sqrt(vhat) + eps)
        print("theta_old: {0}".format(theta_old))
        print("theta_new: {0}".format(theta_new))
        
        theta_new[theta_new < 0.0] = 0.0
        
        if np.sqrt(np.inner(theta_new - theta_old, theta_new - theta_old)) < tolx:

            if verbose:
                end = time.time()
                print("Running time: {}\n".format(end - start))
            return theta_new
        
        print("fold : {0}".format(fold))
        fnew = _func(theta_new, pij, sj, N)
        print("fnew : {0}".format(fnew))
        if np.abs(fold - fnew) < tolf:
            print("Here")
            if verbose:
                end = time.time()
                print("Running time: {}\n".format(end - start))
            return theta_new
        
        theta_old = theta_new
        fold = fnew
        mold = mnew
        vold = vnew
        
                
        if verbose:
            print("{0}th iteration \t theta: {1} obj func: {2} \t grad: {3}"
                  .format(niter, theta_old, fold, g))
            
    print("EXCEED THE MAXIMUM ITERATION NUMBERS!")
    if verbose:
        end = time.time()
        print("Running time : {}\n".format(end - start))
        
    return theta_new

In [41]:
res = adam(lamb0, pij, sj, N, verbose = True)

iteration: 0
theta_old: [0.10189193 0.245029   0.3931284  0.03099201 0.01369657 0.9591778
 0.7693404  0.43347735 0.03013112 0.95252669]
theta_new: [0.10289193 0.246029   0.3921284  0.03199201 0.01469657 0.9581778
 0.7683404  0.43247735 0.03113112 0.95152669]
fold : 3881.6754639209394
fnew : 3877.7404236683747
1th iteration 	 theta: [0.10289193 0.246029   0.3921284  0.03199201 0.01469657 0.9581778
 0.7683404  0.43247735 0.03113112 0.95152669] obj func: 3877.7404236683747 	 grad: [-389.8052538  -160.73676249   16.94476993 -289.41076058 -339.49850239
  418.1189968   690.2313663   330.490323   -488.86005167  812.5258749 ]
iteration: 1
theta_old: [0.10289193 0.246029   0.3921284  0.03199201 0.01469657 0.9581778
 0.7683404  0.43247735 0.03113112 0.95152669]
theta_new: [0.10389193 0.24702866 0.39113044 0.03299201 0.01569657 0.9571778
 0.7673404  0.43147743 0.03213112 0.95052669]
fold : 3877.7404236683747
fnew : 3873.8109515264637
2th iteration 	 theta: [0.10389193 0.24702866 0.39113044 0.0329

theta_old: [0.15938943 0.30216156 0.40613442 0.08813539 0.07100175 0.90120239
 0.71152926 0.37733276 0.08771566 0.89452725]
theta_new: [0.1603712  0.30311388 0.40697215 0.08909688 0.07197767 0.90020459
 0.71053716 0.37643774 0.0886975  0.89352769]
fold : 3661.8744495589553
fnew : 3658.2505524669423
59th iteration 	 theta: [0.1603712  0.30311388 0.40697215 0.08909688 0.07197767 0.90020459
 0.71053716 0.37643774 0.0886975  0.89352769] obj func: 3658.2505524669423 	 grad: [-367.8052538  -141.73676249   -7.05523007 -256.41076058 -312.49850239
  416.1189968   675.2313663   247.490323   -463.86005167  810.5258749 ]
iteration: 59
theta_old: [0.1603712  0.30311388 0.40697215 0.08909688 0.07197767 0.90020459
 0.71053716 0.37643774 0.0886975  0.89352769]
theta_new: [0.16135277 0.30406516 0.40780527 0.09005677 0.0729527  0.89920706
 0.7095452  0.37554598 0.08967911 0.89252828]
fold : 3658.2505524669423
fnew : 3654.629925720812
60th iteration 	 theta: [0.16135277 0.30406516 0.40780527 0.09005677 0

 0.66133775 0.33613352 0.13724366 0.84362632]
theta_new: [0.20985457 0.3493929  0.44093358 0.13674014 0.12090148 0.84960303
 0.66036988 0.33543522 0.13819767 0.84262821]
fold : 3485.0034028388163
fnew : 3481.712304334421
110th iteration 	 theta: [0.20985457 0.3493929  0.44093358 0.13674014 0.12090148 0.84960303
 0.66036988 0.33543522 0.13819767 0.84262821] obj func: 3481.712304334421 	 grad: [-347.8052538  -117.73676249   -3.05523007 -227.41076058 -294.49850239
  410.1189968   645.2313663   165.490323   -439.86005167  809.5258749 ]
iteration: 110
theta_old: [0.20985457 0.3493929  0.44093358 0.13674014 0.12090148 0.84960303
 0.66036988 0.33543522 0.13819767 0.84262821]
theta_new: [0.21080815 0.35024983 0.44141809 0.13764129 0.1218455  0.84861376
 0.65940261 0.33474046 0.13915143 0.84163007]
fold : 3481.712304334421
fnew : 3478.425519641848
111th iteration 	 theta: [0.21080815 0.35024983 0.44141809 0.13764129 0.1218455  0.84861376
 0.65940261 0.33474046 0.13915143 0.84163007] obj func: 3

theta_old: [0.26213236 0.39533302 0.46406572 0.1853586  0.17226386 0.79444111
 0.6068472  0.30324173 0.19105034 0.78691887]
theta_new: [0.26304212 0.39611682 0.46465872 0.18619416 0.17313221 0.79346528
 0.60590664 0.30281426 0.19197604 0.78592958]
fold : 3306.823971078219
fnew : 3303.896084359979
167th iteration 	 theta: [0.26304212 0.39611682 0.46465872 0.18619416 0.17313221 0.79346528
 0.60590664 0.30281426 0.19197604 0.78592958] obj func: 3303.896084359979 	 grad: [-320.8052538   -98.73676249   -4.05523007 -198.41076058 -255.49850239
  400.1189968   611.2313663    80.490323   -412.86005167  798.5258749 ]
iteration: 167
theta_old: [0.26304212 0.39611682 0.46465872 0.18619416 0.17313221 0.79346528
 0.60590664 0.30281426 0.19197604 0.78592958]
theta_new: [0.26395155 0.39689874 0.46526264 0.18702763 0.173999   0.79248965
 0.6049669  0.30239298 0.19290082 0.78494038]
fold : 3303.896084359979
fnew : 3300.972564386068
168th iteration 	 theta: [0.26395155 0.39689874 0.46526264 0.18702763 0.

fnew : 3150.441505835349
222th iteration 	 theta: [0.31241914 0.43681598 0.49112407 0.2305281  0.21956822 0.74026449
 0.55547815 0.2845227  0.24197999 0.73171484] obj func: 3150.441505835349 	 grad: [-293.8052538   -84.73676249   -2.05523007 -180.41076058 -232.49850239
  387.1189968   562.2313663    39.490323   -382.86005167  787.5258749 ]
iteration: 222
theta_old: [0.31241914 0.43681598 0.49112407 0.2305281  0.21956822 0.74026449
 0.55547815 0.2845227  0.24197999 0.73171484]
theta_new: [0.31329162 0.43752645 0.49164169 0.23132044 0.22039072 0.73930856
 0.55458613 0.28427272 0.24286638 0.73073379]
fold : 3150.441505835349
fnew : 3147.8008347090504
223th iteration 	 theta: [0.31329162 0.43752645 0.49164169 0.23132044 0.22039072 0.73930856
 0.55458613 0.28427272 0.24286638 0.73073379] obj func: 3147.8008347090504 	 grad: [-293.8052538   -84.73676249   -1.05523007 -180.41076058 -232.49850239
  387.1189968   562.2313663    38.490323   -382.86005167  787.5258749 ]
iteration: 223
theta_old: 

...

theta_old: [0.35657795 0.27292245 0.25466452 0.13097806 0.14256457 0.05781095
 0.         0.         0.3572456  0.        ]
theta_new: [ 0.35647728  0.27291162  0.25457106  0.13092918  0.1425252   0.0577701
 -0.00098158 -0.0004552   0.3571898  -0.00090965]
fold : 2355.0764572147546
fnew : 2355.0748971301887
2191th iteration 	 theta: [0.35647728 0.27291162 0.25457106 0.13092918 0.1425252  0.0577701
 0.         0.         0.3571898  0.        ] obj func: 2355.0748971301887 	 grad: [ 8.19474620e+00  2.63237508e-01 -5.52300690e-02  2.58923942e+00
  1.50149761e+00  5.11899680e+00  2.31231366e+02  1.64903230e+01
  6.13994833e+00  3.28525875e+02]
iteration: 2191
theta_old: [0.35647728 0.27291162 0.25457106 0.13092918 0.1425252  0.0577701
 0.         0.         0.3571898  0.        ]
theta_new: [ 0.35637664  0.27290082  0.25449061  0.13087996  0.14248742  0.05772869
 -0.00098165 -0.00045576  0.35713397 -0.00090983]
fold : 2355.0748971301887
fnew : 2355.0733351150493
2192th iteration 	 theta: [

theta_old: [0.35142303 0.27229884 0.25287791 0.12825751 0.14060215 0.05584667
 0.         0.         0.35476486 0.        ]
theta_new: [ 0.35134434  0.27228796  0.25290904  0.12822206  0.14056896  0.05581384
 -0.00099114 -0.00052655  0.35472594 -0.00092052]
fold : 2355.0127082313875
fnew : 2355.0118136428937
2249th iteration 	 theta: [0.35134434 0.27228796 0.25290904 0.12822206 0.14056896 0.05581384
 0.         0.         0.35472594 0.        ] obj func: 2355.0118136428937 	 grad: [ 6.19474620e+00  2.63237508e-01 -5.52300690e-02  1.58923942e+00
  1.50149761e+00  4.11899680e+00  2.33231366e+02  1.84903230e+01
  4.13994833e+00  3.30525875e+02]
iteration: 2249
theta_old: [0.35134434 0.27228796 0.25290904 0.12822206 0.14056896 0.05581384
 0.         0.         0.35472594 0.        ]
theta_new: [ 0.35126568  0.27227708  0.2529409   0.12818684  0.14053667  0.05578042
 -0.00099117 -0.00052678  0.35468701 -0.00092063]
fold : 2355.0118136428937
fnew : 2355.0109185255023
2250th iteration 	 theta

  4.13994833e+00  3.31525875e+02]
iteration: 2305
theta_old: [0.34818166 0.27087234 0.25287705 0.12632371 0.13907056 0.0541718
 0.         0.         0.35236496 0.        ]
theta_new: [ 0.34815043  0.27084807  0.2528486   0.12628967  0.1390458   0.05414452
 -0.00099954 -0.00059324  0.35232466 -0.00092762]
fold : 2354.9769577115994
fnew : 2354.9765441761033
2306th iteration 	 theta: [0.34815043 0.27084807 0.2528486  0.12628967 0.1390458  0.05414452
 0.         0.         0.35232466 0.        ] obj func: 2354.9765441761033 	 grad: [ 2.19474620e+00  1.26323751e+00 -5.52300690e-02  1.58923942e+00
  1.50149761e+00  2.11899680e+00  2.35231366e+02  2.04903230e+01
  4.13994833e+00  3.31525875e+02]
iteration: 2306
theta_old: [0.34815043 0.27084807 0.2528486  0.12628967 0.1390458  0.05414452
 0.         0.         0.35232466 0.        ]
theta_new: [ 0.34811945  0.27082082  0.25282691  0.12625561  0.13902102  0.05411792
 -0.00099959 -0.00059378  0.35228437 -0.00092772]
fold : 2354.9765441761033
f

theta_old: [0.34647672 0.26953587 0.25278779 0.12431817 0.13761454 0.0528723
 0.         0.         0.35023162 0.        ]
theta_new: [ 0.34644732  0.26952237  0.25278242  0.12428303  0.13759355  0.05285709
 -0.00100356 -0.0006655   0.35019991 -0.00093181]
fold : 2354.956633088291
fnew : 2354.9563672862832
2364th iteration 	 theta: [0.34644732 0.26952237 0.25278242 0.12428303 0.13759355 0.05285709
 0.         0.         0.35019991 0.        ] obj func: 2354.9563672862832 	 grad: [ 2.19474620e+00  2.63237508e-01 -5.52300690e-02  1.58923942e+00
  5.01497614e-01  2.11899680e+00  2.36231366e+02  2.24903230e+01
  3.13994833e+00  3.31525875e+02]
iteration: 2364
theta_old: [0.34644732 0.26952237 0.25278242 0.12428303 0.13759355 0.05285709
 0.         0.         0.35019991 0.        ]
theta_new: [ 0.34641791  0.26950465  0.25278165  0.12424787  0.1375755   0.05284128
 -0.00100362 -0.00066589  0.35016821 -0.0009319 ]
fold : 2354.9563672862832
fnew : 2354.95610046595
2365th iteration 	 theta: [0

theta_old: [0.34477529 0.26880492 0.25205768 0.12231931 0.1368803  0.0521127
 0.         0.         0.34841066 0.        ]
theta_new: [ 0.34474497  0.26880172  0.25206092  0.12228404  0.13686897  0.05210055
 -0.00100794 -0.00067889  0.34837826 -0.00093572]
fold : 2354.9423443973815
fnew : 2354.9421033571653
2421th iteration 	 theta: [0.34474497 0.26880172 0.25206092 0.12228404 0.13686897 0.05210055
 0.         0.         0.34837826 0.        ] obj func: 2354.9421033571653 	 grad: [ 2.19474620e+00 -7.36762492e-01 -5.52300690e-02  1.58923942e+00
  1.50149761e+00  1.11899680e+00  2.37231366e+02  2.24903230e+01
  3.13994833e+00  3.31525875e+02]
iteration: 2421
theta_old: [0.34474497 0.26880172 0.25206092 0.12228404 0.13686897 0.05210055
 0.         0.         0.34837826 0.        ]
theta_new: [ 0.34471462  0.2687931   0.25206803  0.12224865  0.13685789  0.05208949
 -0.00100794 -0.00067909  0.34834583 -0.00093578]
fold : 2354.9421033571653
fnew : 2354.9418605541687
2422th iteration 	 theta:

In [43]:
objective_lamb(lamb0, pij)

2133.1804709847215

In [44]:
objective_lamb(res, pij)

2354.0039682493616

In [42]:
checkConstraint(res, pij, sj, N)

array([-1.19474620e+00, -2.63237508e-01,  1.05523007e+00, -1.58923942e+00,
       -5.01497614e-01, -1.11899680e+00, -2.40231366e+02, -2.24903230e+01,
       -1.13994833e+00, -3.32525875e+02])

In [45]:
import mip

I = range(N)
V = range(M)

model = mip.Model()

x = [[model.add_var(var_type = mip.BINARY) for j in V] for i in I]

In [46]:
model.objective = mip.maximize(mip.xsum(pij[i][j] * x[i][j] for i in I for j in V))

# local constraints, only 3 benefits recommended for 
for i in I:
    model += mip.xsum(x[i][j] for j in V) == 3
    
# global constraints, the coupons are limited by numbers
for j in V:
    model += mip.xsum(x[i][j] for i in I) <= 3 * sj[j] * N

In [ ]:
model.optimize()

In [7]:
model.objective_value